In [1]:
import numpy as np
import cv2
import SimpleITK as sitk
from matplotlib import pyplot as plt
import scipy.io as io

# path = 'E:\\split_seq_OAI\\reference.mha'
path = '/home/liangzx/code/OAI-registration/newsubject/reference.mha'
image = sitk.ReadImage(path)
resacleFilter = sitk.RescaleIntensityImageFilter()
resacleFilter.SetOutputMaximum(255)
resacleFilter.SetOutputMinimum(0)
img3D = resacleFilter.Execute(image)
img = sitk.GetArrayFromImage(img3D).astype('uint8')
img_tmp = cv2.cvtColor(img[:,:,70],cv2.COLOR_GRAY2RGB)

In [105]:
path = '/home/liangzx/code/OAI-registration/newsubject/reference_f.mha'
fimage = sitk.ReadImage(path)

In [ ]:
mask = np.zeros(img_tmp.shape[:2],np.uint8)
bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)
rect = (50,50,450,290)#划定区域
cv2.grabCut(img_tmp,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)#函数返回值为mask,bgdModel,fgdModel
mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')#0和2做背景

img_tmp = img_tmp*mask2[:,:,np.newaxis]#使用蒙板来获取前景区域



cv2.imshow('p',img_tmp)
cv2.waitKey(0)

In [2]:
import scipy.io as scio

In [3]:
dataFile = '/home/liangzx/code/OAI-registration/newsubject/tibia_mask.mat'
data = scio.loadmat(dataFile)

In [4]:
mask_npy = data['result_new']

In [5]:
fixed_image = img3D

In [6]:
mask_image = sitk.GetImageFromArray(mask_npy)
mask_image.SetDirection(fixed_image.GetDirection())
mask_image.SetOrigin(fixed_image.GetOrigin())
mask_image.SetSpacing(fixed_image.GetSpacing())

In [7]:
def tmp_resample(fixed_image):
    isoresample = sitk.ResampleImageFilter()
#     isoresample.SetDefaultPixelValue(255);
    isoresample.SetInterpolator(sitk.sitkNearestNeighbor)
    isoresample.SetOutputDirection(fixed_image.GetDirection())
    isoresample.SetOutputOrigin(fixed_image.GetOrigin())
    isoresample.SetOutputSpacing(fixed_image.GetSpacing())
    isoresample.SetSize(fixed_image.GetSize())
    return isoresample.Execute(fixed_image)

In [8]:
resacleFilter = sitk.RescaleIntensityImageFilter()
resacleFilter.SetOutputMaximum(255)
resacleFilter.SetOutputMinimum(0)
mask_tmp2 = resacleFilter.Execute(mask_image)

In [9]:
mask_tmp = tmp_resample(mask_tmp2)

In [10]:
# binaryFilter = sitk.BinaryThresholdImageFilter()
# binaryFilter.SetLowerThreshold(0.0)
# binaryFilter.SetUpperThreshold(50.0)
# binaryFilter.SetOutsideValue(255)
# binaryFilter.SetInsideValue(0)
# mask_tmp3 = binaryFilter.Execute(mask_tmp)

In [10]:
from ipywidgets import interact, fixed

In [11]:
def display_images_with_mask(image_z, fixed, moving):
    # img = (1.0 - alpha)*fixed[:,:,image_z] + alpha*moving[:,:,image_z]
    fixed = sitk.GetArrayFromImage(fixed)
    moving = sitk.GetArrayFromImage(moving) # mask
    dst = fixed[image_z,:,:]*0.5+moving[image_z,:,:]*0.5
    plt.imshow(dst, cmap=plt.cm.Greys_r)
    # plt.imshow(sitk.GetArrayViewFromImage(img),cmap=plt.cm.Greys_r);
    plt.axis('off')
    plt.show()

def show_masked_images(image, mask):
    interact(display_images_with_mask, image_z=(0,image.GetSize()[2] - 1), 
             fixed = fixed(image), moving=fixed(mask));

def display_mask(image_z, fixed):
    # img = (1.0 - alpha)*fixed[:,:,image_z] + alpha*moving[:,:,image_z]
    fixed = sitk.GetArrayFromImage(fixed)
    dst = fixed[image_z,:,:]*255
#     plt.figure(figsize=(8, 8))
    plt.imshow(dst, cmap=plt.cm.Greys_r)
    # plt.imshow(sitk.GetArrayViewFromImage(img),cmap=plt.cm.Greys_r);
    plt.axis('off')
    plt.show()
    
def show_mask(image):
    interact(display_mask, image_z=(0,image.GetSize()[2] - 1), fixed = fixed(image));

In [12]:
show_mask(mask_tmp)

interactive(children=(IntSlider(value=191, description='image_z', max=383), Output()), _dom_classes=('widget-i…

In [13]:
resacleFilter = sitk.RescaleIntensityImageFilter()
resacleFilter.SetOutputMaximum(1)
resacleFilter.SetOutputMinimum(0)
mask_tmp = resacleFilter.Execute(mask_tmp)
sitk.WriteImage(sitk.Cast(mask_tmp, sitk.sitkInt16), "/home/liangzx/code/OAI-registration/newsubject/reference_t.mha")